In [2]:
import pandas as pd
import numpy as np
import warnings
import pyodbc
import random
import math
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.optimize import minimize
import os

warnings.filterwarnings(action = 'ignore')

In [3]:
conn_pcor = pyodbc.connect('driver={Oracle in OraClient18Home1};dbq=PCOR;uid=EF0SEL;pwd=EF0SEL#076')
conn_quant = pyodbc.connect('driver={SQL Server};server=46.2.90.172;database=quant;uid=index;pwd=samsung@00')
conn_wisefn = pyodbc.connect('driver={SQL Server};server=46.2.90.172;database=wisefn;uid=index;pwd=samsung@00')

#### 지수 구성종목

In [3]:
start_dt = 19991231
end_dt = 20231020
sql = f'''
        SELECT T1.*
        FROM
        (
            SELECT D2.*, CONCAT('A',BM.STK_CD) COMP_CD, BM.MKT_TYP, BM.KS200_TYP, BM.KOSDAQ150_YN KQ150_TYP
            FROM
            (
                SELECT D.BASE_D, LEAD(D.BASE_D,1) OVER (ORDER BY BASE_D) NXT_D
                FROM  
                (
                SELECT distinct MAX(TRD_DT) BASE_D, EOMONTH(TRD_DT) EOM
                FROM WISEFN..TS_STK_DAILY
                WHERE TRD_DT <= EOMONTH(TRD_DT)
                AND STK_CD = '005930'
                GROUP BY EOMONTH(TRD_DT)
                ) D
            ) D2

            LEFT OUTER JOIN 
            (
                SELECT *
                FROM WISEFN..TS_STK_ISSUE
                WHERE 1 = 1
                AND TRD_STOP_TYP = 0
                AND (MKT_TYP = 1 OR KS200_TYP = 1 OR KOSDAQ150_YN = 1)
                
            ) BM
            ON D2.BASE_D = BM.TRD_DT
        ) T1
        
        INNER JOIN 
        (
            SELECT * FROM WISEFN..TS_STOCK
            WHERE 1 = 1
            AND STK_TYP = 1
            AND ISSUE_TYP = 1
        ) F
        ON RIGHT(T1.COMP_CD, 6) = F.STK_CD
        WHERE T1.BASE_D > {start_dt}
        AND T1.NXT_D <= {end_dt}
        
        ORDER BY T1.BASE_D ASC
        
    
    '''
    
df = pd.read_sql(sql, conn_wisefn)
df.fillna(0, inplace=True)

In [4]:
df.to_json('231020_index_constituent.json')

#### 가격데이터

In [5]:
start_dt = 19991231
end_dt = 20230927

sql_p = f'''
        
        SELECT D2.BASE_D, CONCAT('A',A.STK_CD) COMP_CD, A.VAL PRC
            FROM
            (
                SELECT D.BASE_D, LEAD(D.BASE_D,1) OVER (ORDER BY BASE_D) NXT_D
                FROM  
                (
                SELECT distinct MAX(TRD_DT) BASE_D, EOMONTH(TRD_DT) EOM
                FROM WISEFN..TS_STK_DAILY
                WHERE TRD_DT <= EOMONTH(TRD_DT)
                AND STK_CD = '005930'
                GROUP BY EOMONTH(TRD_DT)
                ) D
            ) D2
            
            LEFT OUTER JOIN 
            (
                SELECT A.*
                FROM WISEFN..TS_STK_DATA A
                WHERE A.ITEM_CD = '100300' AND A.ITEM_TYP = 'S' AND A.TRD_DT > {start_dt} AND A.TRD_DT <= {end_dt}
            ) A
            
            ON D2.BASE_D = A.TRD_DT
            
            ORDER BY D2.BASE_D ASC
            '''

    
df = pd.read_sql(sql_p, conn_wisefn)
df.dropna(axis=0, inplace=True)

In [6]:
df.to_json('230927_stk_prc.json')

#### 가격데이터_데일리

In [5]:
start_dt = 20041231
end_dt = 20230927

sql_p = f'''
        SELECT TRD_DT, CONCAT('A',STK_CD) COMP_CD, VAL PRC
        FROM WISEFN..TS_STK_DATA 
        WHERE 1=1 
        AND ITEM_CD = '100300' 
        AND ITEM_TYP = 'S' 
        AND TRD_DT > {start_dt} AND TRD_DT <= {end_dt}
        ORDER BY TRD_DT ASC

        '''

    
df = pd.read_sql(sql_p, conn_wisefn)
df.dropna(axis=0, inplace=True)

In [6]:
df.to_json('230927_stk_prc_daily.json')

#### 시가총액 데이터

In [7]:
start_dt = 19991231
end_dt = 20230927

sql_p = f'''
        
        SELECT D2.BASE_D, CONCAT('A',A.STK_CD) COMP_CD, A.VAL MKTCAP
            FROM
            (
                SELECT D.BASE_D, LEAD(D.BASE_D,1) OVER (ORDER BY BASE_D) NXT_D
                FROM  
                (
                SELECT distinct MAX(TRD_DT) BASE_D, EOMONTH(TRD_DT) EOM
                FROM WISEFN..TS_STK_DAILY
                WHERE TRD_DT <= EOMONTH(TRD_DT)
                AND STK_CD = '005930'
                GROUP BY EOMONTH(TRD_DT)
                ) D
            ) D2
            
            LEFT OUTER JOIN 
            (
                SELECT A.*
                FROM WISEFN..TS_STK_DATA A
                WHERE A.ITEM_CD = '102100' AND A.ITEM_TYP = 'S' AND A.TRD_DT > {start_dt} AND A.TRD_DT <= {end_dt}
            ) A
            
            ON D2.BASE_D = A.TRD_DT
            
            ORDER BY D2.BASE_D ASC
            '''

    
df = pd.read_sql(sql_p, conn_wisefn)
df.dropna(axis=0, inplace=True)

In [8]:
df.to_json('230927_stk_mktcap.json')

#### Turnover

In [9]:
start_dt = 19991231
end_dt = 20230927

sql = f'''
    SELECT A1.TRD_DT, A1.STK_CD, A1.LISTED_STK, A2.FFR, A3.TRD_STK
    FROM (
        SELECT TRD_DT, STK_CD, VAL LISTED_STK
        FROM TS_STK_DATA
        WHERE 1=1
        AND ITEM_CD = '101500'
        ) A1

    LEFT OUTER JOIN (
                    SELECT TRD_DT, STK_CD, VAL FFR
                    FROM TS_STK_DATA
                    WHERE 1=1
                    AND ITEM_CD = '102060'
                    ) A2

    ON A1.STK_CD = A2.STK_CD
    AND A1.TRD_DT = A2.TRD_DT

    LEFT OUTER JOIN (
                    SELECT TRD_DT, STK_CD, VAL TRD_STK
                    FROM TS_STK_DATA
                    WHERE 1=1
                    AND ITEM_CD = '100900'
                    ) A3

    ON A1.STK_CD = A3.STK_CD
    AND A1.TRD_DT = A3.TRD_DT

    WHERE A1.TRD_DT > {start_dt}
    AND A1.TRD_DT <= {end_dt}
    '''

df = pd.read_sql(sql, conn_wisefn)
df.sort_values('TRD_DT', inplace=True)

In [10]:
df.sort_values('TRD_DT', inplace=True)

In [11]:
df

,TRD_DT,STK_CD,LISTED_STK,FFR,TRD_STK
5540,20000104,000100,4906120.0,NaN,77320.0
357071,20000104,013890,3430343.0,94.61,44420.0
213134,20000104,005940,92502345.0,NaN,2330680.0
505047,20000104,026220,7200000.0,66.66,100288.0
272660,20000104,008970,2303176.0,NaN,2200.0
...,...,...,...,...,...
12769575,20230927,570091,1000000.0,NaN,89.0
12769576,20230927,580026,500000.0,NaN,2825.0
12769577,20230927,580027,500000.0,NaN,187.0
12769426,20230927,570053,1000000.0,NaN,79.0


In [12]:
df_turnover = df.copy()
df_turnover = df_turnover.sort_values(['STK_CD','TRD_DT'], ascending = [True, True]).reset_index(drop=True)
df_turnover['FFR'] = [np.nan if x > 100 or x <= 0 else x for x in df_turnover['FFR']]
indexer = df_turnover.groupby('STK_CD', as_index=False).nth(0).index

df_turnover.loc[indexer,'FFR'] = [x if x >= 0 else -100 for x in df_turnover.loc[indexer,'FFR']]
df_turnover = df_turnover.ffill().replace(-100, np.nan)

indexer = df_turnover.groupby('STK_CD', as_index=False).nth(-1).index

df_turnover.loc[indexer,'FFR'] = [x if x >= 0 else -100 for x in df_turnover.loc[indexer,'FFR']]
df_turnover = df_turnover.bfill().replace(-100, np.nan)

df_turnover.dropna(axis=0, inplace=True)

df_turnover['FFR_STK'] = df_turnover['LISTED_STK']*df_turnover['FFR']/100
df_turnover['key'] = [x[:6] for x in df_turnover['TRD_DT']]
temp = df_turnover.groupby(['STK_CD','key'])['TRD_STK'].sum().reset_index()
temp.columns = ['STK_CD','key','TRD_STK_m']
df_turnover = df_turnover.merge(temp, how = 'left', on = ['STK_CD', 'key'])
df_turnover['TO'] = df_turnover['TRD_STK_m']/df_turnover['FFR_STK']

df_turnover_2 = df_turnover[['TRD_DT','STK_CD','TO']]
df_turnover_2.columns = ['BASE_D','COMP_CD','TO']
df_turnover_2['COMP_CD'] = ['A'+x for x in df_turnover_2['COMP_CD']]

In [13]:
df_turnover_2.to_json('230927_stk_turnover.json')

#### 섹터정보

In [14]:
start_dt = 19991231
end_dt = 20230927

sql_sector = f'''
                SELECT U3.BASE_D, U3.COMP_CD, B2.SECTOR_NM
                FROM
                    (
                    SELECT D.BASE_D, CONCAT('A',U2.STK_CD) COMP_CD, U2.GICS_CD
                    FROM
                        (
                        SELECT distinct MAX(TRD_DT) BASE_D, EOMONTH(TRD_DT) EOM
                        FROM WISEFN..TS_STK_DAILY
                        WHERE TRD_DT <= EOMONTH(TRD_DT)
                        AND STK_CD = '005930'
                        GROUP BY EOMONTH(TRD_DT)
                        ) D
                        INNER JOIN 

                        (SELECT U1.*, B1.GICS_CD
                        FROM
                            (SELECT U0.*, B0.CMP_CD
                            FROM WISEFN..TS_STK_ISSUE U0
                            LEFT OUTER JOIN WISEFN..TS_STOCK B0
                            ON U0.STK_CD = B0.STK_CD
                            WHERE 1=1
                            AND U0.TRD_DT >= {start_dt}
                            AND U0.TRD_DT <= {end_dt}
                            AND U0.TRD_STOP_TYP <> 1) U1

                            LEFT OUTER JOIN WISEFN..TC_COMPANY B1
                            ON U1.CMP_CD = B1.CMP_CD) U2

                    ON D.BASE_D = U2.TRD_DT
                ) U3
                
                LEFT OUTER JOIN 
                    (SELECT SEC_CD SECTOR_CD, SEC_N_KOR SECTOR_NM
                    FROM WISEFN..TC_SECTOR
                    WHERE SEC_TYP = 'G'
                        AND LEN(SEC_CD) = 3
                    ) B2
                ON LEFT(U3.GICS_CD,3) = B2.SECTOR_CD
                
                ORDER BY U3.BASE_D ASC
                
            '''



df = pd.read_sql(sql_sector, conn_wisefn)

In [15]:
df.to_json('230927_stk_sector.json')